In [1]:
import os
import json
import time
import boto3
import tweepy
import pprint
import datetime
import numpy as np
import pandas as pd
import botocore.session
from boto3.dynamodb.conditions import Key, Attr

with open("../creds.json", "r") as f:
    creds = json.load(f)
    f.close()

client = boto3.client('dynamodb',
  endpoint_url = "http://xanaxprincess.asuscomm.com:8001",
  aws_access_key_id=creds["aws-access-key"],
  aws_secret_access_key=creds["aws-secret-access-key"],
  region_name='us-east-2')

dynamodb = boto3.resource('dynamodb', \
            endpoint_url = "http://xanaxprincess.asuscomm.com:8001", \
            region_name='us-east-2')
    
print("imported modules.")

ModuleNotFoundError: No module named 'boto3'

In [2]:
query = """
select * from tweets
where
group by
order by
"""

print("query: {}".format(query))

query: 
select * from tweets
where
group by
order by



In [3]:
print(client)

In [4]:
print(dynamodb)

dynamodb.ServiceResource()
